In [1]:
import numpy as np
import pandas as pd
from scipy import signal
from scipy.stats import chi2
import chart_studio.plotly as py
import cufflinks as cf
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

%matplotlib inline

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode (connected=True)
cf.go_offline()



In [2]:
#IMPORTING DATA
ano=2020 #measurement year

#wind farm
parque=pd.read_csv("wind farm {}.csv".format(ano),delimiter="\t") 
parque["Data"]=pd.to_datetime(parque["Data"],dayfirst=True)
parque.set_index("Data",inplace= True)


#C4
cdvc4=pd.read_csv("c4 {}.csv".format(ano),delimiter="\t")
cdvc4["Data"]=pd.to_datetime(cdvc4["Data"],dayfirst=True)
cdvc4.set_index("Data",inplace= True)

#C6
cdvc6=pd.read_csv("c6 {}.csv".format(ano),delimiter="\t")
cdvc6["Data"]=pd.to_datetime(cdvc6["Data"],dayfirst=True)
cdvc6.set_index("Data",inplace= True)

#C5
cdvc5=pd.read_csv("c5 {}.csv".format(ano),delimiter="\t") 
cdvc5["Data"]=pd.to_datetime(cdvc5["Data"],dayfirst=True)
cdvc5.set_index("Data",inplace= True)


#C5+C6
x=cdvc5['Potência Ativa Total Méd.']+cdvc6['Potência Ativa Total Méd.']
cdvc5c6=x.to_frame().dropna()


#C5+C4
x=cdvc5['Potência Ativa Total Méd.']+cdvc4['Potência Ativa Total Méd.']
cdvc5c4=x.to_frame().dropna()

#C6+C4
x=cdvc6['Potência Ativa Total Méd.']+cdvc4['Potência Ativa Total Méd.']
cdvc6c4=x.to_frame().dropna()

#C5+C6+C4
x=cdvc5c6['Potência Ativa Total Méd.']+cdvc4['Potência Ativa Total Méd.']
cdvc5c6c4=x.to_frame().dropna()

In [3]:
#----------PARAMETERS-----------------------
#welch function
fs=1/.25 # sampling frequency
tam_bloco=2400 #block segmentation for welch function
tapper='boxcar' #window function
ov=tam_bloco//2 #overlap


#confidence interval
overlap_fraction=ov/tam_bloco #overlap fraction for cvanfidence interval
cl=0.05 #confidence interval

#Block segmentation for pandas
eq='10T' # 10 minutes block segmentation for pandas


# Data selection
data_start="2020-08-14 00:00:00.000" #starting date
data_end="2020-08-14 23:59:59.999" #ending date
lista=[cdvc5,cdvc6,cdvc4,cdvc5c6,cdvc6c4,cdvc5c4,cdvc5c6c4,parque] #dataset list
legenda=['7 Turbines-C5','8 Turbines-C6','7 Turbines-C4','15 Turbines-C5+C6','14 Turbines-C5+C4',
        '15 Turbines-C6+C4','22 Turbines - C5+C6+C4','51 Turbines - Wind Farm'] #legend for the plot

#other
op=False# Data normalization option

#---------STORAGE VARIABLES-------------------------------
psd_dataframe=pd.DataFrame(columns=legenda) #PSD
pot_utilizada_lista=[] # Wind power data used for PSD list
pot_utilizada_dataframe=pd.DataFrame(columns=legenda) # Wind power data used for PSD 
var_psd_lista=[] #PSD variance list 
var_psd_dataframe=pd.DataFrame(columns=legenda) #PSD variance
lower_bound_dataframe=pd.DataFrame(columns=legenda) #lower bound confidence inveval
upper_bound_dataframe=pd.DataFrame(columns=legenda) #Upper bound confidence inveval



#----------CODE-------------------------------------------
for numero,dataset in enumerate(lista):
    mask=(dataset.index>=data_start) & (dataset.index<=data_end)
    
    if op:
  
        y=dataset[mask]["Potência Ativa Total Méd."]/10**6
        p=y/(y.mean())
    
    else:
        p=dataset[mask]["Potência Ativa Total Méd."]/10**6
    q=p
           
    #PSD
    pot_utilizada_lista.append(q)
    pot_utilizada_dataframe[legenda[numero]]=p
    q=q-q.mean()
    freq,psd=signal.welch(q.ravel(),fs,nperseg=tam_bloco,noverlap=ov,window=tapper,nfft=tam_bloco,detrend='constant',scaling='density')
    
    #Confidence interval
    k_eff=(len(q)/tam_bloco)*(1/1-overlap_fraction)
    dof=2*k_eff
    chi2_lower = chi2.ppf(cl / 2, dof)
    chi2_upper = chi2.ppf(1 - cl / 2, dof)
    lower_bound = dof * psd / chi2_upper
    upper_bound = dof * psd / chi2_lower
    
    
    #Result Storage
    psd_dataframe[legenda[numero]]=psd
    lower_bound_dataframe[legenda[numero]]=lower_bound
    upper_bound_dataframe[legenda[numero]]=upper_bound
    var_psd_lista.append(psd.sum()*(freq[1]-freq[0]))

    
    
    
psd_dataframe.set_index(freq,inplace=True)
lower_bound_dataframe.set_index(freq,inplace=True)
upper_bound_dataframe.set_index(freq,inplace=True)
var_psd=pd.DataFrame(var_psd_lista,index=legenda,columns=['var PSD']).T
var_st=(pot_utilizada_dataframe.resample(eq).var(ddof=0).mean()).to_frame(name='var ST').T
var_dataframe=pd.concat([var_st,var_psd])



#----------PLOTS------------------------------------------- 

fig = go.Figure()


for legenda_nome in legenda:
    # PSD
    fig.add_trace(go.Scatter(
        x=psd_dataframe.index[1:],
        y=psd_dataframe[legenda_nome].iloc[1:],
        mode='lines',
        name=f'PSD {legenda_nome}'
    ))

    # Confidence Interval
    fig.add_trace(go.Scatter(
        x=np.concatenate([psd_dataframe.index[1:],psd_dataframe.index[:0:-1]]),
        y=np.concatenate([lower_bound_dataframe[legenda_nome].iloc[1:], upper_bound_dataframe[legenda_nome].iloc[:0:-1]]),
        fill='toself',
        fillcolor='rgba(0, 123, 255, 0.2)',
        line=dict(color='rgba(255,255,255,0)'),
        hoverinfo="skip",
        showlegend=True,  # don't add legend for each fill
        name=f'95% CI {legenda_nome}'
    ))


fig.update_xaxes(type='log', title_text='Frequency (Hz)')
fig.update_yaxes(type='log', title_text='$MW^{2}/Hz$')


fig.update_layout(
    title='PSD with 95% Confidence Interval',
    template='plotly_white',
    legend=dict(x=1.05, y=1),  # move legend outside
    margin=dict(l=40, r=40, t=40, b=40)
)

# fig.show()
